In [2]:
#import libraries, dependencies

#Using a CSV file with all zip codes in Denver from:
#https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
#selected only zip codes in Denver City using excel

#TODO: import CSV and create a data frame

#TODO: select parameters for request to geoapify, which will find all healthcare facilities in each zip code

#TODO: Use a for loop to make an api request for each zip code, and place the healthcare facility results into a DF

#TODO: Draw a map which locates each healthcare facility (dot size population of the zip code? color by type of facility?)

#TODO: get population of each zip code and add to data frame with healthcare facility info 

#TODO: plot #of healthcare facilities per zip code against population per zip code (double bars with zip as x axis?)


#TODO: count # facilities with wheelchair access
#TODO: If not very close to 100%, plot types of facility against wheelchair access

#TODO: decide on at least four more visualizations to create:
        

#TODO: written analysis

#TODO: prepare slides for presentation

#TODO: practice presentation(or at least be clear on respective roles)

In [ ]:
import pandas as pd
import numpy as pd
import requests
import json
from config import api_key
api_key

In [13]:
url = 'https://api.geoapify.com/v2/places'
latitude = -33.8698439
longitude = 151.2082848
radius = 5000
filters = f'circle: {longitude}, {latitude}', {radius}
bias = f"proximity:{longitude},{latitude}"
limit = 5
parameters = {
    'categories': "healthcare",
    "limit": limit,
    "filters":filters,
    "bias": bias,
    "apiKey":api_key}

results = requests.get(url , params=parameters)    
results.json()       


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Chemist Warehouse',
    'country': 'Australia',
    'country_code': 'au',
    'state': 'New South Wales',
    'city': 'Sydney',
    'municipality': 'Council of the City of Sydney',
    'postcode': '2000',
    'suburb': 'Sydney',
    'street': 'Strand Arcade',
    'lon': 151.2080349,
    'lat': -33.8696525,
    'state_code': 'NSW',
    'formatted': 'Chemist Warehouse, Strand Arcade, Sydney NSW 2000, Australia',
    'address_line1': 'Chemist Warehouse',
    'address_line2': 'Strand Arcade, Sydney NSW 2000, Australia',
    'categories': ['healthcare', 'wheelchair', 'wheelchair.yes'],
    'details': ['details', 'details.facilities'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Chemist Warehouse',
      'brand': 'Chemist Warehou

In [12]:
print(results.url)

https://api.geoapify.com/v2/places?categories=healthcare&limit=5&filter=circle%3A+151.2082848%2C+-33.8698439&filter=5000&bias=proximity%3A151.2082848%2C-33.8698439&apiKey=10df7cccac6c43b08ab640bcf5b7e4a9
